In [56]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle as pkl

GENERATED_JSON_PATH = '../mongo/employees.json'


In [57]:
employee_data = pd.read_json(GENERATED_JSON_PATH)
target_features_fields = employee_data[['salary','role','work_location']]
target_features_fields = target_features_fields.dropna()
target_features_fields = target_features_fields.drop_duplicates()

print(target_features_fields.head())

   salary                  role work_location
0   50000  Marketing Specialist      New York
1  120000        Data Scientist      New York
2  120000        Data Scientist      Hartford
3   95000            HR Manager      Hartford
4   55000            HR Manager       Chicago


In [58]:
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = encoder.fit_transform(target_features_fields[['role','work_location']])
X_encoded = pd.DataFrame(X_encoded)
y= target_features_fields['salary']
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(accuracy)




0.5594879238245223


In [ ]:
with open('model_dump.pkl', 'wb') as f:
    pkl.dump(model, f)